In [4]:
from crapstack.document_stores.faiss import FAISSDocumentStore


In [5]:
document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str="Flat")


ValueError: The number of documents in the SQL database (3904) doesn't match the number of embeddings in FAISS (0). Make sure your FAISS configuration file points to the same database that you used when you saved the original index.

In [3]:
import pandas as pd

data = pd.read_csv('c:/users/kchun/desktop/notebooks/patents_v6.csv')
data

,patent_id,mainclass_id,year,patent_text
0,3930300,257,1976,Junction field effect transistor A junction field effect transistor V shape...
1,3930303,257,1976,Method manufacturing compact thermoelectric modules A manufacturing method t...
2,3930304,257,1976,Method apparatus selective burnout trimming integrated circuit units Supplem...
3,3930305,257,1976,Method manufacturing integrated circuits A method manufacturing integrated c...
4,3930306,257,1976,Process attaching lead member semiconductor device A thermally electrically ...
...,...,...,...,...
7814692,8496534,470,2013,Group taps prepared hole cutting tools In FIG 2 diameter Dd drilling edges d...
7814693,8534022,470,2013,Twisted threaded reinforcing bar Techniques reinforcing concrete using rebar...
7814694,8550755,470,2013,Tap driver rigidsynchronous tapping Disclosed tap driver tap driver system r...
7814695,8602696,470,2013,Form tap plurality lobes A form tap tapping article may include longitudinal...


In [4]:
from crapstack.haystack.schema import Document

documents = [Document(content=txt) for txt in data['patent_text'].sample(frac=.0005)]
document_store.write_documents(documents=documents)

Writing Documents: 10000it [00:03, 2979.62it/s]            


In [3]:
from crapstack.haystack.nodes.retriever import DensePassageRetriever
from crapstack.haystack.nodes.answer_generator.transformers import Seq2SeqGenerator
from crapstack.haystack.pipelines.standard_pipelines import GenerativeQAPipeline
# from transformers import T5ForConditionalGeneration
# import torch

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
    passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
)

generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")
# generator = T5ForConditionalGeneration.from_pretrained("google/flan-ul2", torch_dtype=torch.bfloat16, device_map="auto")                                                                 

pipe = GenerativeQAPipeline(generator, retriever)

NameError: name 'document_store' is not defined

In [ ]:
document_store.update_embeddings(retriever)